In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM
from langchain import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain
from langchain.memory import ConversationSummaryBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from IPython.core.display import display, HTML

c:\Users\jebcu\anaconda3\envs\databricks\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\jebcu\AppData\Local\Temp\ipykernel_17908\3916475486.py:9: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
if not torch.cuda.is_available():
    print("Running dolly without GPU will be slow.")

In [3]:
workplace_vector_db_path = "C:\\Users\\jebcu\\Desktop\\SYTYCH_Hackathon\\workplace_db"
 
hf_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
chroma_db = Chroma(collection_name="workplace_docs", embedding_function=hf_embed, persist_directory=workplace_vector_db_path)

Using embedded DuckDB with persistence: data will be stored in: C:\Users\jebcu\Desktop\SYTYCH_Hackathon\workplace_db


In [4]:
def build_qa_chain():
  torch.cuda.empty_cache()
  # Defining our prompt content.
  # langchain will load our similar documents as {context}
  template = """You are a chatbot having a conversation with a human. Your are asked to answer career questions, and you are helping the human applying for jobs.
  Given the following extracted parts of a long document and a question, answer the user question. If you don't know, say that you do not know. 
  
  {context}
 
  {chat_history}
 
  {human_input}
 
  Response:
  """
  prompt = PromptTemplate(input_variables=['context', 'human_input', 'chat_history'], template=template)
 
  # Increase max_new_tokens for a longer response
  # Other settings might give better results! Play around
  model_name = "databricks/dolly-v2-3b" # can use dolly-v2-3b, dolly-v2-7b or dolly-v2-12b for smaller model and faster inferences.
  instruct_pipeline = pipeline(model=model_name, torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto", 
                               return_full_text=True, max_new_tokens=256, top_p=0.95, top_k=50)
  hf_pipe = HuggingFacePipeline(pipeline=instruct_pipeline)
 
  # Add a summarizer to our memory conversation
  # Let's make sure we don't summarize the discussion too much to avoid losing to much of the content
 
  # Models we'll use to summarize our chat history
  # We could use one of these models: https://huggingface.co/models?filter=summarization. facebook/bart-large-cnn gives great results, we'll use t5-small for memory
  summarize_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small", device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True)
  summarize_tokenizer = AutoTokenizer.from_pretrained("t5-small", padding_side="left", model_max_length = 512)
  pipe_summary = pipeline("summarization", model=summarize_model, tokenizer=summarize_tokenizer) #, max_new_tokens=500, min_new_tokens=300
  # langchain pipeline doesn't support summarization yet, we added it as temp fix in the companion notebook _resources/00-init 
  hf_summary = HuggingFacePipeline(pipeline=pipe_summary)
  #will keep 500 token and then ask for a summary. Removes prefix as our model isn't trained on specific chat prefix and can get confused.
  memory = ConversationSummaryBufferMemory(llm=hf_summary, memory_key="chat_history", input_key="human_input", max_token_limit=500, human_prefix = "", ai_prefix = "")
 
  # Set verbose=True to see the full prompt:
  print("loading chain, this can take some time...")
  return load_qa_chain(llm=hf_pipe, chain_type="stuff", prompt=prompt, verbose=True, memory=memory)


In [5]:
def displayHTML(html):
  display(HTML(html))

In [6]:
class ChatBot():
  def __init__(self, db):
    self.reset_context()
    self.db = db
 
  def reset_context(self):
    self.sources = []
    self.discussion = []
    # Building the chain will load Dolly and can take some time depending on the model size and your GPU
    self.qa_chain = build_qa_chain()
    displayHTML("<h1>Hi! I'm a chat bot specialized in the workplace. How Can I help you today?</h1>")
 
  def get_similar_docs(self, question, similar_doc_count):
    return self.db.similarity_search(question, k=similar_doc_count)
 
  def chat(self, question):
    # Keep the last 3 discussion to search similar content
    self.discussion.append(question)
    similar_docs = self.get_similar_docs(" \n".join(self.discussion[-3:]), similar_doc_count=2)
    # Remove similar doc if they're already in the last questions (as it's already in the history)
    similar_docs = [doc for doc in similar_docs if doc.metadata['source'] not in self.sources[-3:]]
 
    result = self.qa_chain({"input_documents": similar_docs, "human_input": question})
    # Cleanup the answer for better display:
    answer = result['output_text'].capitalize()
    result_html = f"<p><blockquote style=\"font-size:24\">{question}</blockquote></p>"
    result_html += f"<p><blockquote style=\"font-size:18px\">{answer}</blockquote></p>"
    result_html += "<p><hr/></p>"
    for d in result["input_documents"]:
      source_id = d.metadata["source"]
      self.sources.append(source_id)
      result_html += f"<p><blockquote>{d.page_content}<br/>(Source: <a href=\"https://workplace.stackexchange.com/a/{source_id}\">{source_id}</a>)</blockquote></p>"
    displayHTML(result_html)

In [7]:
chat_bot = ChatBot(chroma_db)

loading chain, this can take some time...


In [8]:
chat_bot.chat("How should I start looking for jobs?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human. Your are asked to answer career questions, and you are helping the human applying for jobs.
  Given the following extracted parts of a long document and a question, answer the user question. If you don't know, say that you do not know. 
  
  I am in my last year of university and have been looking at where I would like to take my career. So far I have seen a number of graduate jobs on offer (such as on stackoverflow jobs), but I’m not sure whether it is too early to apply, as I have 9~ months until I graduate (then again several of the graduate jobs I’ve seen have been up for a number of months already.)
As a small edit, I actually finish my studies around 2 months before my graduation.
When is the earliest that I should apply for a graduate job in the industry?


No, it is not too early to start applying for jobs, though it may

In [9]:
chat_bot.chat("What should I do to get an interview?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human. Your are asked to answer career questions, and you are helping the human applying for jobs.
  Given the following extracted parts of a long document and a question, answer the user question. If you don't know, say that you do not know. 
  
  
 
  : How should I start looking for jobs?
: 
First of all, make sure you don't miss applying to jobs once they are available. You can start by reading articles from companies, universities, and career forums. You could also search for job posts on sites like LinkedIn and Indeed.com.  Keep in mind that companies may use different applications to apply so you might want to take your resume and apply online with all the applications.
 
  What should I do to get an interview?
 
  Response:
  

> Finished chain.

> Finished chain.


In [10]:
chat_bot.chat("What should I do to prepare for an interview?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human. Your are asked to answer career questions, and you are helping the human applying for jobs.
  Given the following extracted parts of a long document and a question, answer the user question. If you don't know, say that you do not know. 
  
  I've never had a "real" job before. How should I expect the first day?
I'm supposed to check in, but I'm completely oblivious to how this all will pan out.
Do I just show up and follow through? What if no one guides me? Where do you go when first arriving?
I think there's like an employee login room or something? Break room? Not sure.
The job is at a supermarket and it's entry-level/minimum wage. I wasn't given full info or anything -- just told to show up for my first day on Thursday and "report" in but not much else was clarified (I was also a bit nervous too & still am).
What do I expect?

In [11]:
chat_bot.chat("Can you give me some tips?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human. Your are asked to answer career questions, and you are helping the human applying for jobs.
  Given the following extracted parts of a long document and a question, answer the user question. If you don't know, say that you do not know. 
  
  I always hear that "do you have any question" when the intervew are going to end.
Actually, I want to know what should I do for best?
What is the best answer actually?
update
I have seperate 2 stage.
1.During Phone screen Interview.
2.F2F interview.


Short answer: Ask what you want to know!
Ask what you want to know about the company, hours, after hours activities, social club, anything you want to know that would influence your decision to take the job.  Remember, an interview is for both of you to decide if it is a good fit.
You want to see if there are any factors that would make it a go